In [99]:
from tika import parser
import pandas as pd
import re
import env

In [94]:
# parser.from_file('pdfs/2022-09-27_bill.pdf')['content']

In [40]:
def get_data(filePath = 'pdfs/2022-09-27_bill.pdf'):
    '''
    
    '''
    text = parser.from_file(filePath)['content']
    print(f'Text extraction from {filePath} complete.')
    
    return text

In [95]:
# text = get_data()
# text

In [96]:
# test_text = get_data(filePath = 'pdfs/2022-08-25_bill.pdf')
# test_text

## What I want to extract
✔️

Field | Document Field | Description
:-- | :-- | :--
✅`date_bill`| | Billing Date | 
✅`start_bill`| | Billing Start Date |
✅`end_bill` | | Billing End Date
✅`period_bill` | | Length of Billing Period (days)
✅`date_meter` | | Meter Read Date
✅`meter`| | Current Meter Read |
✅`consumption`| | kWh Used |
`service` | Service Availability Charge | Covers the cost of metering and billing for your address, regardless of consumption
`amt_kwh` | Energy Charge per kWh
`amt_total` | Total Energy Charge | Recovers costs for power plants and other infrastructure based on the amount of electricity you use
`fuel_kwh` |
`fuel_total` | | The cost for fuel above the base rate, and it fluctuates monthly based on the prices CPS Energy pays for fuel
`reg_kwh` | 
`reg_total` | | Shows the state mandated fees and costs associated with the Electric Reliability Council of Texas (ERCOT) grid
✅`peak` | | kWh Used in Excess of 600kWh | Applies during the months of June through September when an additional charge is applied for every kilowatt hour (kWh) used in excess of 600 kWh.
`peak_kwh` | 
`peak_total` | | The Peak Capacity Charge covers the higher costs for electricity CPS Energy incurs during summer months when demand for electricity is at its highest. 
`city_services` | | Fees CPS Energy collects for the City of San Antonio’s Solid Waste Management Department 

# Parsing Text Using Indices

In [6]:
def find_text(str_start, str_end, text_ = text):
    '''
    
    '''
    start_index = text_.index(str_start) + len(str_start)
    end_index = text_.index(str_end, start_index)
    
    return text_[start_index:end_index]

## 1. Billing Date

In [100]:
date_bill_beg = f'7{env.zip}\n\n'
date_bill_end = '\n\nOn or Before'

AttributeError: module 'env' has no attribute 'zip'

In [97]:
# re.findall(rf'{date_bill_beg}', text)

In [9]:
find_text(date_bill_beg, date_bill_end)

'09/27/22'

In [84]:
find_text(date_bill_beg, date_bill_end, test_text)

'08/25/22'

In [87]:
df_bill_date = pd.DataFrame([find_text(date_bill_beg, date_bill_end), find_text(date_bill_beg, date_bill_end, test_text)], \
             columns = ['billing_date'])
df_bill_date

,billing_date
0,09/27/22
1,08/25/22


### Billing Date Function

In [10]:
def get_bill_date(str_start = f'{env.zip}\n\n', str_end = '\n\nOn or Before', text = text):
    '''
    This function returns the billing data from the bill text as a string, by looking at the index value of the 
    text that comes before and after the value.
    '''
    start_index = text.index(str_start) + len(str_start)
    end_index = text.index(str_end, start_index)
    
    return text[start_index:end_index]

In [11]:
get_bill_date()

'09/27/22'

## 2. Start Bill, End Bill, and Length of Bill Period

In [12]:
# bill_period_start = 'Billing Period '
# bill_period_end = '\nYour next'

In [13]:
# bill_parse = find_text(bill_period_start, bill_period_end)
# bill_parse

In [14]:
# find_text(bill_period_start, bill_period_end, text = test_text)

### Parsing out Start Bill, End Bill, and Length of Bill Period

In [15]:
# beg_bill_date = ''.join(re.findall(r'^[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', bill_parse))
# beg_bill_date

In [16]:
# !pip install arrow
import arrow

In [17]:
# bill_start = arrow.get(beg_bill_date, 'MMM DD, YYYY ').format('YYYY-MM-DD')
# bill_start

In [18]:
# bill_end_date = ''.join(re.findall(r'-[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', bill_parse))
# bill_end_date

In [19]:
# bill_end = arrow.get(bill_end_date, '- MMM DD, YYYY').format('YYYY-MM-DD')
# bill_end

In [20]:
# pd.to_datetime(bill_end) - pd.to_datetime(bill_start)

In [21]:
import datetime as dt

In [22]:
# bill_days = (pd.to_datetime(bill_end) - pd.to_datetime(bill_start)).days + 1
# bill_days

In [80]:
def get_bill_period(text_ = text, str_start = 'Billing Period ', str_end = '\nYour next'):
    '''
    
    '''
    bill_parse = parse_text(str_start, str_end, text_)
    print(f'Parsed Bill Period: {bill_parse}')
    
    bill_start = ''.join(re.findall(r'^[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', bill_parse))
    bill_start = arrow.get(bill_start, 'MMM DD, YYYY ').format('YYYY-MM-DD')
    print(f'Parsed Bill Start: {bill_start}')
    
    bill_end = ''.join(re.findall(r'-[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', bill_parse))
    bill_end = arrow.get(bill_end, '- MMM DD, YYYY').format('YYYY-MM-DD')
    print(f'Parsed Bill End: {bill_end}')
    
    bill_period = (pd.to_datetime(bill_end) - pd.to_datetime(bill_start)).days + 1
    print(f'Parsed Bill Period Duration: {bill_period} days')
    print()
    
    return bill_start, bill_end, bill_period

In [81]:
get_bill_period()

Parsed Bill Period: Aug 24, 2022 - Sep 23, 2022
Parsed Bill Start: 2022-08-24
Parsed Bill End: 2022-09-23
Parsed Bill Period Duration: 31 days



('2022-08-24', '2022-09-23', 31)

In [82]:
get_bill_period(text_ = test_text)

Parsed Bill Period: Jul 26, 2022 - Aug 23, 2022
Parsed Bill Start: 2022-07-26
Parsed Bill End: 2022-08-23
Parsed Bill Period Duration: 29 days



('2022-07-26', '2022-08-23', 29)

In [88]:
df_bill_period = pd.DataFrame([get_bill_period(), get_bill_period(text_ = test_text)], \
             columns = ['bill_start', 'bill_end', 'bill_days'])
df_bill_period

Parsed Bill Period: Aug 24, 2022 - Sep 23, 2022
Parsed Bill Start: 2022-08-24
Parsed Bill End: 2022-09-23
Parsed Bill Period Duration: 31 days

Parsed Bill Period: Jul 26, 2022 - Aug 23, 2022
Parsed Bill Start: 2022-07-26
Parsed Bill End: 2022-08-23
Parsed Bill Period Duration: 29 days



,bill_start,bill_end,bill_days
0,2022-08-24,2022-09-23,31
1,2022-07-26,2022-08-23,29


In [91]:
pd.concat([df_bill_date, df_bill_period], axis = 1)

,billing_date,bill_start,bill_end,bill_days
0,09/27/22,2022-08-24,2022-09-23,31
1,08/25/22,2022-07-26,2022-08-23,29


## 3. Meter and Consumption

In [48]:
find_text('(R-', '#6271330')

'09/23/2022) 7996378953 1,010'

In [54]:
# current meter reading
find_text('(R-', '#6271330')[12:17]

'79963'

In [59]:
int(find_text('(R-', '#6271330')[12:17])

79963

In [52]:
# meter read date
find_text('(R-', '#6271330')[:10]

'09/23/2022'

In [56]:
arrow.get(find_text('(R-', '#6271330')[:10], 'MM/DD/YYYY').format('YYYY-MM-DD')

'2022-09-23'

In [53]:
# consumption (kWh)
find_text('(R-', '#6271330')[-5:]

'1,010'

In [58]:
int((find_text('(R-', '#6271330')[-5:]).replace(',', ''))

1010

In [60]:
# peak consumption
int((find_text('(R-', '#6271330')[-5:]).replace(',', '')) - 600

410

In [68]:
def get_consumption(text_ = text):
    '''
    
    '''
    data = find_text('(R-', '#6271330', text_)
    
    # current meter reading
    meter = find_text('(R-', '#6271330', text_)[12:17]
    meter = int(meter)
    
    # meter read date
    meter_date = find_text('(R-', '#6271330', text_)[:10]
    meter_date = arrow.get(meter_date, 'MM/DD/YYYY').format('YYYY-MM-DD')
    
    # consumption (kWh)
    consumption = find_text('(R-', '#6271330', text_)[-5:]
    consumption = int(consumption.replace(',', ''))
    
    # peak consumption (kWh)
    peak = consumption - 600
    
    return meter_date, consumption, peak

In [69]:
get_consumption()

('2022-09-23', 1010, 410)

In [70]:
get_consumption(text_ = test_text)

('2022-08-23', 1164, 564)

In [92]:
df_con = pd.DataFrame([get_consumption(), get_consumption(text_ = test_text)], \
             columns = ['meter_date', 'consumption', 'peak'])
df_con

,meter_date,consumption,peak
0,2022-09-23,1010,410
1,2022-08-23,1164,564


In [93]:
pd.concat([df_bill_date, df_bill_period, df_con], axis = 1)

,billing_date,bill_start,bill_end,bill_days,meter_date,consumption,peak
0,09/27/22,2022-08-24,2022-09-23,31,2022-09-23,1010,410
1,08/25/22,2022-07-26,2022-08-23,29,2022-08-23,1164,564
